<a href="https://colab.research.google.com/github/freddy-7/TI3002C/blob/main/4_Valores_at%C3%ADpicos_DataFrame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Importamos las librerias pandas, numpy y matplotlib respectivamente
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#Carga desde un archivo .csv sin indice
data = pd.read_csv('Tokyo_Limpia (1).csv')

In [4]:
#Verificamos información del DataFrame
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22783 entries, 0 to 22782
Data columns (total 50 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            22783 non-null  int64  
 1   name                                          22783 non-null  object 
 2   description                                   22783 non-null  object 
 3   neighbourhood_cleansed                        22783 non-null  object 
 4   latitude                                      22783 non-null  float64
 5   longitude                                     22783 non-null  float64
 6   property_type                                 22783 non-null  object 
 7   room_type                                     22783 non-null  object 
 8   accommodates                                  22783 non-null  int64  
 9   bathrooms                                     22783 non-null 

In [5]:
#Corroboramos valores nulos
print("Nulos iniciales:\n", data.isnull().sum())

Nulos iniciales:
 id                                              0
name                                            0
description                                     0
neighbourhood_cleansed                          0
latitude                                        0
longitude                                       0
property_type                                   0
room_type                                       0
accommodates                                    0
bathrooms                                       0
bedrooms                                        0
beds                                            0
amenities                                       0
price                                           0
minimum_nights                                  0
maximum_nights                                  0
availability_30                                 0
availability_60                                 0
availability_90                                 0
availability_365                

**PROCEDIMIENTO PARA ELIMINAR VALORES NULOS EN DATAFRAME**

In [6]:
# Reemplazo valores nulos con backfill
data1 = data.fillna(method="bfill")
print("\nNulos después de bfill:\n", data1.isnull().sum())


Nulos después de bfill:
 id                                              0
name                                            0
description                                     0
neighbourhood_cleansed                          0
latitude                                        0
longitude                                       0
property_type                                   0
room_type                                       0
accommodates                                    0
bathrooms                                       0
bedrooms                                        0
beds                                            0
amenities                                       0
price                                           0
minimum_nights                                  0
maximum_nights                                  0
availability_30                                 0
availability_60                                 0
availability_90                                 0
availability_365        

/tmp/ipython-input-2031588718.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data1 = data.fillna(method="bfill")


In [9]:
# Divido el DataFrame en cuantitativas (cols 1 a 22) y cualitativa (col 0)
cuantitativas = data1.select_dtypes(include="number")

cualitativas = data1.select_dtypes(exclude="number")


In [12]:
import matplotlib.pyplot as plt
from ipywidgets import Dropdown, Checkbox, interact

# --- Selección de numéricas excluyendo ids y derivados ---
excluir = [
    "id", "host_id",
    "calculated_host_listings_count",
    "calculated_host_listings_count_entire_homes",
    "calculated_host_listings_count_private_rooms",
    "calculated_host_listings_count_shared_rooms"
]

cuantitativas = data1.select_dtypes(include="number").drop(columns=excluir, errors="ignore")
cualitativas = data1.drop(columns=cuantitativas.columns)

# Dropdown de columnas
col_selector = Dropdown(options=list(cuantitativas.columns), description='Columna:')

# Opciones extra
show_std = Checkbox(value=False, description='Límites 3σ')
show_iqr = Checkbox(value=False, description='Límites IQR')

def plot_box(col, show_std_lines, show_iqr_lines):
    serie = cuantitativas[col].dropna()
    plt.figure(figsize=(12,4))
    plt.boxplot(serie, vert=False, showfliers=True)
    plt.title(f"Boxplot: {col}")
    plt.xlabel(col)

    if show_std_lines:
        mu, sd = serie.mean(), serie.std()
        plt.axvline(mu-3*sd, linestyle='--')
        plt.axvline(mu, linestyle='-')
        plt.axvline(mu+3*sd, linestyle='--')

    if show_iqr_lines:
        Q1, Q3 = serie.quantile([0.25,0.75])
        IQR = Q3 - Q1
        plt.axvline(Q1-1.5*IQR, linestyle=':')
        plt.axvline(Q1, linestyle='-.')
        plt.axvline(Q3, linestyle='-.')
        plt.axvline(Q3+1.5*IQR, linestyle=':')

    plt.show()
    print("Resumen estadístico:\n", serie.describe())

interact(plot_box, col=col_selector, show_std_lines=show_std, show_iqr_lines=show_iqr);



interactive(children=(Dropdown(description='Columna:', options=('latitude', 'longitude', 'accommodates', 'bath…

In [13]:

# ============================================================
# MÉTODO DESVIACIÓN ESTÁNDAR (3σ)
# ============================================================
y = cuantitativas.copy()
mu = y.mean()
sd = y.std()
lim_sup = mu + 3*sd
lim_inf = mu - 3*sd

data3 = y.apply(lambda col: col.where((col <= lim_sup[col.name]) & (col >= lim_inf[col.name])))
imputados_3s_por_col = (data3.isna().sum() - y.isna().sum()).clip(lower=0)
total_imputados_3s = int(imputados_3s_por_col.sum())

data_clean_std = data3.fillna(round(data3.mean(), 1))
Datos_limpios_STD = pd.concat([cualitativas, data_clean_std], axis=1)
Datos_limpios_STD.to_csv("Datos_limpios_STD.csv", index=False)

In [14]:

# ============================================================
# MÉTODO RANGO INTERCUARTÍLICO (IQR)
# ============================================================
Q1 = y.quantile(0.25)
Q3 = y.quantile(0.75)
IQR = Q3 - Q1
lim_sup_iqr = Q3 + 1.5*IQR
lim_inf_iqr = Q1 - 1.5*IQR

data3_iqr = y.apply(lambda col: col.where((col <= lim_sup_iqr[col.name]) & (col >= lim_inf_iqr[col.name])))
imputados_iqr_por_col = (data3_iqr.isna().sum() - y.isna().sum()).clip(lower=0)
total_imputados_IQR = int(imputados_iqr_por_col.sum())

data_clean_iqr = data3_iqr.fillna(round(data3_iqr.mean(), 1))
Datos_limpios_IQR = pd.concat([cualitativas, data_clean_iqr], axis=1)
Datos_limpios_IQR.to_csv("Datos_limpios_IQR.csv", index=False)

In [15]:

# ============================================================
# COMPARATIVO
# ============================================================
nfilas = y.shape[0]
ncols_num = y.shape[1]

pct_imp_3s  = (total_imputados_3s  / (nfilas * ncols_num)) * 100
pct_imp_iqr = (total_imputados_IQR / (nfilas * ncols_num)) * 100

print("\n================= RESUMEN COMPARATIVO =================")
print(f"Outliers detectados (3σ):  {total_imputados_3s}  | % sobre celdas numéricas: {pct_imp_3s:.3f}%")
print(f"Outliers detectados (IQR): {total_imputados_IQR} | % sobre celdas numéricas: {pct_imp_iqr:.3f}%")
print("Archivos generados: Datos_limpios_STD.csv  y  Datos_limpios_IQR.csv")
print("=======================================================\n")

# Top columnas afectadas
print("Top 10 columnas más afectadas por 3σ:\n", imputados_3s_por_col.sort_values(ascending=False).head(10))
print("\nTop 10 columnas más afectadas por IQR:\n", imputados_iqr_por_col.sort_values(ascending=False).head(10))

# Filas afectadas
filas_afectadas_3s  = (data3.isna().any(axis=1)).sum()
filas_afectadas_IQR = (data3_iqr.isna().any(axis=1)).sum()
print("\nFilas afectadas 3σ:", filas_afectadas_3s)
print("Filas afectadas IQR:", filas_afectadas_IQR)


================= RESUMEN COMPARATIVO =================
Outliers detectados (3σ):  8032  | % sobre celdas numéricas: 1.175%
Outliers detectados (IQR): 42886 | % sobre celdas numéricas: 6.275%
Archivos generados: Datos_limpios_STD.csv  y  Datos_limpios_IQR.csv

Top 10 columnas más afectadas por 3σ:
 latitude                       703
availability_30                645
host_acceptance_rate           527
review_scores_communication    479
accommodates                   474
bathrooms                      447
number_of_reviews              379
review_scores_value            344
review_scores_rating           343
review_scores_cleanliness      341
dtype: int64

Top 10 columnas más afectadas por IQR:
 bathrooms                    4193
host_response_rate           3588
minimum_nights               3495
maximum_nights               2407
host_total_listings_count    2098
latitude                     2039
number_of_reviews            1997
host_acceptance_rate         1980
host_listings_count    

In [16]:
explicacion = """
=== Decisión sobre el tratamiento de outliers ===

Se compararon dos métodos:

1) Desviación Estándar (3σ)
   - Detectó 8,032 celdas atípicas (≈1.17% del total numérico).
   - Afectó 4,707 filas.
   - Identificó principalmente valores extremos en variables continuas
     como latitude, availability_30 y métricas de review_scores.
   - El impacto es moderado y conserva la mayoría de los datos válidos.

2) Rango Intercuartílico (IQR, k=1.5)
   - Detectó 42,886 celdas atípicas (≈6.28% del total numérico).
   - Afectó 17,464 filas.
   - Señaló como outliers valores plausibles en variables discretas o sesgadas
     (ej. bathrooms, minimum_nights, maximum_nights).
   - Este enfoque resultó demasiado agresivo y podría distorsionar información real.

Conclusión:
El criterio de 3σ es más adecuado para este dataset, ya que mantiene un balance
entre la detección de valores extremos y la conservación de información.
Se recomienda usar la base generada con este método (Datos_limpios_STD.csv)
como la versión principal para análisis posteriores. La base con IQR (Datos_limpios_IQR.csv)
se conserva solo como referencia exploratoria.
"""

print(explicacion)



=== Decisión sobre el tratamiento de outliers ===

Se compararon dos métodos:

1) Desviación Estándar (3σ)
   - Detectó 8,032 celdas atípicas (≈1.17% del total numérico).
   - Afectó 4,707 filas.
   - Identificó principalmente valores extremos en variables continuas
     como latitude, availability_30 y métricas de review_scores.
   - El impacto es moderado y conserva la mayoría de los datos válidos.

2) Rango Intercuartílico (IQR, k=1.5)
   - Detectó 42,886 celdas atípicas (≈6.28% del total numérico).
   - Afectó 17,464 filas.
   - Señaló como outliers valores plausibles en variables discretas o sesgadas
     (ej. bathrooms, minimum_nights, maximum_nights).
   - Este enfoque resultó demasiado agresivo y podría distorsionar información real.

Conclusión:
El criterio de 3σ es más adecuado para este dataset, ya que mantiene un balance
entre la detección de valores extremos y la conservación de información.
Se recomienda usar la base generada con este método (Datos_limpios_STD.csv)
como 